In [1]:
import pandas as pd
import time
from dotenv import load_dotenv
load_dotenv("./.env.local")
import os
BEARER_TOKEN = os.getenv("BEARER_TOKEN")
import tweepy
client = tweepy.Client(BEARER_TOKEN)

In [2]:
full_dataset = pd.read_csv('UkraineCombinedTweetsDeduped20220227-131611.csv.gzip', compression='gzip')

C:\Users\Bao\AppData\Local\Temp\ipykernel_13572\3161298840.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  full_dataset = pd.read_csv('UkraineCombinedTweetsDeduped20220227-131611.csv.gzip', compression='gzip')


In [11]:
df_en = full_dataset[full_dataset['language']=='en']
df_en_filteted = df_en[["userid", "tweetid", "text", "hashtags"]]
df_no_duplicate = df_en_filteted.drop_duplicates(subset='text', keep='first')

print(f"{len(full_dataset)=}")
print(f"{len(df_en_filteted)=}")
print(f"{len(df_no_duplicate)=}")

df_sample = df_no_duplicate.head(2)

len(full_dataset)=1229349
len(df_en_filteted)=723256
len(df_no_duplicate)=205571


In [14]:
edgelist_retweet = open(f"edgelists/edgelist_retweet_{time.time()}.txt", "a")
edgelist_retweet.write("tweet_id rt_user_id\n")

edgelist_like = open(f"edgelists/edgelist_like_{time.time()}.txt", "a")
edgelist_like.write("tweet_id like_user_id\n")

for index, row in df_sample.iterrows():
    tweetid = row['tweetid']
    tweet = client.get_tweet(id=tweetid, expansions='referenced_tweets.id')
    og_tweet_id = tweet.includes['tweets'][0].id

    users_retweet = client.get_retweeters(id=og_tweet_id)
    for user in users_retweet.data:
        edgelist_retweet.write(f"{og_tweet_id} {user.id}\n")
        continue

    next_token = users_retweet.meta.get('next_token')

    n = 0
    while(next_token and n > 2):
        users_retweet = client.get_retweeters(id=og_tweet_id, next_token=next_token)
        for user in users_retweet.data:
            edgelist_retweet.write(f"{og_tweet_id} {user.id}\n")
            continue
        next_token = users_retweet.meta.get('next_token')
    
    users_like = client.get_liking_users(id=og_tweet_id)
    for user in users_like.data:
        edgelist_like.write(f"{og_tweet_id} {user.id}\n")
        continue
    n = n + 1 
    next_token = users_like.meta.get('next_token')

    n = 0
    while(next_token and n > 2):
        users_like = client.get_retweeters(id=og_tweet_id, next_token=next_token)
        for user in users_like.data:
            edgelist_like.write(f"{og_tweet_id} {user.id}\n")
            continue
        n = n + 1 
        next_token = users_like.meta.get('next_token')

edgelist_retweet.close()
edgelist_like.close()

FileNotFoundError: [Errno 2] No such file or directory: '/edgelists/edgelist_retweet_1648542199.5365098.txt'